In [32]:

from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [124]:
fileFPath='/home/mprakash/Projects/enron-experiments/significant-terms/output/firstpass/maildir/delainey-d/maildir_delainey-d_all_documents_428.'


In [125]:
f = open(fileFPath, 'r')
content = f.read()
f.close()
# print 'File Content'
print content

Janet, get rid of these LM's - too many cooks!
Regards
Delainey
	Enron North America Corp.
	
	From:  Tammy R Shepperd                           08/08/2000 03:17 PM
	

To: David W Delainey/HOU/ECT@ECT
cc:  
Subject: Master Turbine Lists

As I mentioned to you earlier, there is much interest/concern about managing 
the ENE portfolio of turbines, both from a balance sheet and a deal 
perspective, as a result I've had numerous data requests in the last few 
weeks including one today from RAC.

I thought you might be interested in the comment I have highlighted below, in 
blue, regarding the LM6000's.

Tammy



Fred L Kelly@ENRON_DEVELOPMENT
08/03/2000 03:09 PM
To: Mike Coleman/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT
cc: Dick Westfahl/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, fkelly@flash.net.com, 
Dan Shultz/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Matthew 
Tezyk/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT, Bill Williams@ECT, David 
Parquet/SF/ECT@ECT, Wayne Mays@ECT, Mathew Gimble@ECT, Chip 
Schneider/NA/Enro

In [5]:
import spacy
nlp = spacy.load('en')

In [142]:
from itertools import *

set_noise = set('information meeting review group thanks regards'.split())
dict_orgs = {}
dict_people = {}
dict_location = {}
dict_vocab = {}

noise_words = set('information meeting review group thanks regards additional informational to cc bcc am pm enron'.split())
stop_words = set('for a of the and to in what my you this is your where what our'.split())
unwanted_pos = ['SPACE', 'DET', 'NUM', 'PRON']


def drop_as_candidate(noun_chunk):
    # drop when the noun phrase is a single word, and is a pronoun, determiner or other common noise
   
#     tokens = [(token.orth_, token.pos_) for token in noun_chunk]
    tokens = [token for token in noun_chunk]
    if(len(tokens) == 1):
#         print 'token is', tokens[0][0], tokens[0][1]
        if(tokens[0].pos_ in unwanted_pos): return True
        if(tokens[0].orth_ in stop_words): return True
        if(tokens[0].orth_ in noise_words): return True
        return False
    else:
        # if the chunk has any space in POS, it is probably not useful and we can drop it from consideration
#         space_token = [token for token in noun_chunk if token.pos_ == 'SPACE']
#         return len(space_token) > 0
        return False

def drop_as_candidate2(tokens_of_a_chunk):
    # drop when the noun phrase is a single word, and is a pronoun, determiner or other common noise
   
#     tokens = [(token.orth_, token.pos_) for token in noun_chunk]
#     tokens = [token for token in noun_chunk]
    if(len(tokens_of_a_chunk) == 1):
#         print 'token is', tokens[0][0], tokens[0][1]
        if(tokens_of_a_chunk[0].pos_ in unwanted_pos): return True
        if(tokens_of_a_chunk[0].orth_.lower() in stop_words): return True 
        if(tokens_of_a_chunk[0].orth_.lower() in noise_words): return True
        return False
    else:
        return False
        


def convert_multiline_phrase_to_singleline(noun_chunk):
    singleline_tokens = [token for token in noun_chunk if token.pos_ != 'SPACE']
    return singleline_tokens

def convert_multiline_phrase_to_multiple_phrases(noun_chunk):
    '''
Original text looks like this-> 

PUHCA Stand-Alone 

Dave -

FYI.


Raw Tokens look like this -> [(u'PUHCA', u'PROPN'), (u'Stand', u'VERB'), (u'-', u'PUNCT'), (u'Alone', u'PROPN'), (u'\n\n', u'SPACE'), (u'Dave', u'PROPN'), (u'-', u'PUNCT'), (u'\n\n', u'SPACE'), (u'FYI.', u'PROPN')]

Converted tokens (showing here only the words, not the pos tag)> 
[u'PUHCA', u'Stand', u'-', u'Alone']
Converted> [u'Dave', u'-']
Converted> [u'FYI.']

    '''
    multiple_phrases = []
    current_phrase = []
    for token in noun_chunk:
        if token.pos_ != 'SPACE':
            current_phrase.append(token)
        else:
            multiple_phrases.append(current_phrase)
            current_phrase = []
    multiple_phrases.append(current_phrase)
    

            
            
#         current_phrase.append(token) if token.pos_ != 'SPACE' else multiple_phrases.append(current_phrase) current_phrase = []
    return multiple_phrases

def replace_numbers_and_dates(tokens):
#     modified_tokens = [token for token in tokens 
#                   if token.pos_ not in unwanted_pos and token.orth_.lower() not in stop_words]
    
    modified_tokens = [token for token in tokens if token.pos_ not in ['NUM']]
    return modified_tokens
    


def compose_phrase_from_tokens(tokens):
#     X_NO_SPACE = ''
#     X_SINGLE_SPACE = ' '
#     token_string_as_list = [(lambda x: X_NO_SPACE if x.pos_=='PUNCT' else X_SINGLE_SPACE)(token) + token.orth_ for token in tokens]
    token_string_as_list = [token.orth_ if token.pos_ in ['PROPN','NOUN'] else token.orth_.lower() for token in tokens]
    # print 'token string as list->', token_string_as_list
    # if the last token is punctuation, like dash or dot - . then ignore them
    if(token_string_as_list[-1] in ['-','.']):
        token_string_as_list = token_string_as_list[:-1]
    
    result_phrase = ' '.join(token_string_as_list)
    # get rid of leading/trailing space or punctuation chars if any
    result_phrase = result_phrase.strip(' ,.-')
    return result_phrase
    

def remove_unbalanced_quote(phrase):
    corrected_phrase = phrase
    lastsinglequoteindices = [index for index, eachchar in enumerate(phrase) if eachchar=='\'']
    lastdoublequoteindices = [index for index, eachchar in enumerate(phrase) if eachchar=='"']
    
#     if (len(lastsinglequoteindices) % 2 == 1):
#         # this is unbalanced, remove the last quote
        
#         # find position to remove
#         pos = lastsinglequoteindices[-1]
        
#         # remove the character at that position
#         corrected_phrase = phrase[:pos] + phrase[(pos+1):] 
        
    if (len(lastdoublequoteindices) % 2 == 1):
        # this is unbalanced, remove the last quote
        
        # find position to remove
        pos2 = lastdoublequoteindices[-1]
        
        # remove the character at that position
        corrected_phrase = corrected_phrase[:pos2] + corrected_phrase[(pos2+1):] 
    
    return corrected_phrase

def split_commas_into_additional_phrases(phrase):
    subphrases = phrase.split(",")
    return [phrase.strip() for phrase in subphrases]

        
        

def generate_candidate_chunks(doc):
    nchunks = doc.noun_chunks
    candidate_chunks = []

    for nc in nchunks:
        print 'RAW_CHUNK->', [(token.orth_, token.pos_) for token in nc]
    #     print 'RAW_CHUNK_END'
        rectified_tokens1 = convert_multiline_phrase_to_singleline(nc)
        drop_this_chunk = drop_as_candidate(nc)
        if( drop_this_chunk ):
            print 'PROCESSED_CHUNK->', 'Dropped'
            continue
        else:
            rectified_tokens2 = replace_numbers_and_dates(rectified_tokens1)
            reconstructed_phrase = compose_phrase_from_tokens(rectified_tokens2)
            reconstructed_phrase2 = remove_unbalanced_quote(reconstructed_phrase)
            multi_phrase_by_comma_split = split_commas_into_additional_phrases(reconstructed_phrase2)
#             print 'PROCESSED_CHUNK->', compose_phrase_from_tokens(rectified_tokens)

            candidate_chunks.extend(multi_phrase_by_comma_split)
        
    return candidate_chunks

def generate_candidate_chunks2(doc):
    nchunks = doc.noun_chunks
    candidate_chunks = []
    # each chunk is made up of set of spacy tokens
    # were going to see if a chunk itself should better be broken into multiple chunks
    # 
    tokens_for_chunks = []
    
    for nc in nchunks:
        # print 'Original->', nc
        sub_phrases = convert_multiline_phrase_to_multiple_phrases(nc)
        # print 'Expanded->', sub_phrases
        tokens_for_chunks.extend(sub_phrases)
    

    for xchunk in tokens_for_chunks:
        # print 'RAW_CHUNK->', [(xtoken.orth_, xtoken.pos_) for xtoken in xchunk]
    #     print 'RAW_CHUNK_END'
        
        drop_this_chunk = drop_as_candidate2(xchunk)
        if( drop_this_chunk ):
            # print 'PROCESSED_CHUNK->', 'Dropped'
            continue
        else:
            rectified_tokens2 = replace_numbers_and_dates(xchunk)
            if(drop_as_candidate2(rectified_tokens2)):
                # print 'PROCESSED_CHUNK->', 'Dropped'
                continue
            else:
                reconstructed_phrase = compose_phrase_from_tokens(rectified_tokens2)
                reconstructed_phrase2 = remove_unbalanced_quote(reconstructed_phrase)
                multi_phrase_by_comma_split = split_commas_into_additional_phrases(reconstructed_phrase2)
                # print 'PROCESSED_CHUNK->', compose_phrase_from_tokens(rectified_tokens)
                candidate_chunks.extend(multi_phrase_by_comma_split)
        
    return candidate_chunks

def test_chunk_creation_if_space_separated(doc):
    nchunks = doc.noun_chunks
    candidate_chunks = []
    for nc in nchunks:
        print 'Original->', nc
        print 'RAW->',[(token.orth_, token.pos_) for token in nc]
        multiple_phrases = convert_multiline_phrase_to_multiple_phrases(nc)
        for p in multiple_phrases:
            print 'Converted>', [token.orth_ for token in p]
        
    

def examine_sentence_parsing(doc):
    sents = doc.sents
    for sent in sents:
        print 'Sentence->', sent
    
    
def generate_org_entities(doc):
    entities = doc.ents
    org_entities = [e.text for e in entities if e.label_=='ORG']
    for xorg in org_entities:
        # print e.text, e.label_ , [w.tag_ for w in e]
        sanitized_xorg = xorg.strip(',.- ')
        print 'ORG->', sanitized_xorg

    

doc = nlp(unicode(content, "utf-8"))



In [143]:
# examine_sentence_parsing(doc)
print 'uncomment to see parsed sentences'
# test_chunk_creation_if_space_separated(doc)

uncomment to see parsed sentences


In [144]:
candidate_chunks = generate_candidate_chunks2(doc)
for chunk in candidate_chunks:
    print chunk

these LM 's - too many cooks
Delainey
Enron North America Corp
Tammy R Shepperd
David W Delainey/HOU/ECT@ECT
the ENE portfolio
turbines
a balance sheet
a deal
perspective
a result
numerous data requests
the last few
weeks
RAC
the comment
the LM6000
Tammy
Fred L Kelly@ENRON_DEVELOPMENT
Mike Coleman/ENRON_DEVELOPMENT@ENRON_DEVELOPMENT
Subject
Master Turbine
the effort
RAC
an overall Enron
turbine effort
the LM6000
a subset
Rick Buy
Skilling
the board
the " long " turbine position
thus the current
effort
RAC
a more formal process
RAC
a master list
turbines
input
EECC
Dan Shultz 's team
that effort
redundant list efforts
Matt Gimble 's list
the LM6000 effort
his data
the Enron Master Turbine List
Mat Tezyk
dan/mat Tezyk
some turbine lists
manufacturing and QA checks
those lists
the Enron Master Turbine list
finally - Larry Izzo
Dan
a list
ESA
Dan
Matt Gimble 's list
ESA 's needs
summary
redundant efforts
unneccessary work
the note
f.kelly
Mike Coleman
Dick Westfahl
Subject
confusion
everyo

In [145]:
# test entities detection
generate_org_entities(doc)

ORG-> LM
ORG-> Enron
ORG-> ENE
ORG-> RAC
ORG-> Master Turbine Listing  


ORG-> RAC
ORG-> EECC
ORG-> Enron
ORG-> LM6000
ORG-> EECC
ORG-> RAC
ORG-> Enron
ORG-> EECC
ORG-> Enron Master Turbine List
ORG-> Tezyk
ORG-> the Enron Master Turbine
ORG-> ESA
ORG-> ENA
ORG-> RAC
ORG-> ESA
ORG-> Master Turbine Listing
ORG-> ENA/Houston
ORG-> The Master Turbine Listing
ORG-> LM6000
ORG-> LM6000 Timing page
ORG-> the Master Turbine Schedule


In [ ]:
def generate_labels(doc):
    label_candidate = generate_candidate_chunks2(doc)
    